In [1]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from google.colab import output

!mkdir "./train_imgs"
!unzip "/data/train_imgs.zip" -d "./train_imgs"

!mkdir "./test_imgs"
!unzip "/data/test_imgs.zip" -d "./test_imgs"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000019.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000021.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000023.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000025.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000027.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000029.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000031.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000001.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000003.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000005.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000007.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000009.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000011.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000013.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000015.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000017.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000019.j

In [3]:
import pandas as pd
import numpy as np
import random
import cv2
import glob
import random
import warnings    
import matplotlib.pyplot as plt       
from keras_preprocessing.image import ImageDataGenerator
warnings.filterwarnings('ignore')       
%matplotlib inline

In [4]:
train_y = pd.read_csv("./data/train_augumentation.csv")
valid_y = pd.read_csv("./data/valid.csv")
precomputed_train = pd.read_csv("./data/precomputed_train.csv")
precomputed_valid = pd.read_csv("./data/precomputed_valid.csv")
columns = train_y.columns

In [6]:
datagen=ImageDataGenerator(rescale = 1/255) 
test_datagen=ImageDataGenerator(rescale = 1/255) 

train_generator=datagen.flow_from_dataframe(
    dataframe=train_y[:3776],
        directory='train_imgs/',
            x_col="image",
                y_col=columns[1:],
                    batch_size=1,
                        shuffle=False,
                            class_mode="raw",
                                target_size=(1080,1920))
valid_generator=test_datagen.flow_from_dataframe(
    dataframe=valid_y[:],
        directory='train_imgs/',
            x_col="image",
                y_col=columns[1:],
                    batch_size=1,
                        shuffle=False,
                            class_mode="raw",
                                target_size=(1080,1920))

Found 3776 validated image filenames.
Found 419 validated image filenames.


In [7]:
train_y = train_y.values[:,1:].astype(float)
valid_y = valid_y.values[:,1:].astype(float)

In [8]:
precomputed_train = precomputed_train.values[:,1:].astype(float)
precomputed_valid = precomputed_valid.values[:,1:].astype(float)
precomputed_train.shape, precomputed_valid.shape , train_y.shape, valid_y.shape

((32494, 2048), (419, 2048), (32494, 48), (419, 48))

In [9]:
import tensorflow as tf
from tensorflow.keras.datasets.cifar10 import load_data                           
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
my_rnet = tf.keras.applications.ResNet152V2(
    include_top=False, weights='imagenet',input_shape=(1080,1920,3), pooling = "avg")
my_rnet.trainable = False

234553344/234545216 [==============================] - 3s 0us/step


In [12]:
my_rnet.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
my_rnet.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1080, 1920,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 1086, 1926, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 540, 960, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 542, 962, 64) 0           conv1_conv[0][0]                 
________________________________________________________________________________________

In [13]:
#Se propondra la capa densa
topModel = Sequential()
topModel.add(Input((2048,)))
topModel.add(BatchNormalization())

topModel.add(Dense(1024,  activation='relu'))
topModel.add(BatchNormalization())
topModel.add(Dense(1024,  activation='relu'))
topModel.add(BatchNormalization())


topModel.add(Dense(512,  activation='relu'))
topModel.add(BatchNormalization())
topModel.add(Dense(512,  activation='relu'))
topModel.add(BatchNormalization())

topModel.add(Dense(256,  activation='relu'))
topModel.add(BatchNormalization())
topModel.add(Dense(256,  activation='relu'))
topModel.add(BatchNormalization())

topModel.add(Dense(48)) 

topModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2

In [16]:
first_decay_steps = 100
initial_learning_rate = 0.4
lr_decayed_fn = (tf.keras.experimental.
                 CosineDecayRestarts
                 (initial_learning_rate, first_decay_steps))

mc = ModelCheckpoint('best_model.h5', monitor='val_mae', mode='min', verbose=1, save_best_only=True)

topModel.compile(loss='mse', optimizer=Adam(lr_decayed_fn), metrics=['mae'])
topModel.fit(precomputed_train, train_y, epochs=10000, batch_size=1024, validation_data=(precomputed_val, valid_y),callbacks=[mc])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 8751/10000
32/32 [==============================] - 1s 18ms/step - loss: 121.8352 - mae: 8.7312 - val_loss: 2163801893852122430742622451531776.0000 - val_mae: 1764189223780352.0000

Epoch 08751: val_mae did not improve from 6.28420
Epoch 8752/10000
32/32 [==============================] - 1s 19ms/step - loss: 76.7570 - mae: 7.0002 - val_loss: 15609564765001381212567176472428544.0000 - val_mae: 4747333115838464.0000

Epoch 08752: val_mae did not improve from 6.28420
Epoch 8753/10000
32/32 [==============================] - 1s 18ms/step - loss: 84.1877 - mae: 7.3033 - val_loss: 5519195970888800141582283365679104.0000 - val_mae: 2936734891376640.0000

Epoch 08753: val_mae did not improve from 6.28420
Epoch 8754/10000
32/32 [==============================] - 1s 18ms/step - loss: 80.1814 - mae: 7.0304 - val_loss: 7629827034413730305718479749120.0000 - val_mae: 103966511726592.0000

Epoch 08754: val_mae did not improve from 6.28420
Epoch 8755/10000
3

In [53]:
loaded_model = load_model('best_model.h5')
final_model = Sequential([my_rnet, topModel])
final_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Functional)     (None, 2048)              58331648  
_________________________________________________________________
sequential (Sequential)      (None, 48)                4167216   
Total params: 62,498,864
Trainable params: 4,155,952
Non-trainable params: 58,342,912
_________________________________________________________________


In [44]:
test_df = pd.read_csv("./data/sample_submission.csv")
columns = test_df.columns

In [33]:
test_datagen=ImageDataGenerator(rescale = 1/255)
test_generator = test_datagen.flow_from_dataframe(
     dataframe=test_df[:],
         directory='test_imgs',
             x_col="image",
                 y_col=columns[0],
                     batch_size=1,
                         shuffle=False,
                             class_mode="raw",
                                 target_size=(1080,1920))

Found 1600 validated image filenames.


In [ ]:
pred = final_model.predict(test_generator, verbose=1)
test_df.iloc[:,1:] = pred
test_df.to_csv('keypoint_resNet.csv', index=False)